# Load List of Kings and Queens
|  
|  
v  
**!!!!! ERROR WITH Robert Brunner!!!**

In [1]:
import pandas as pd
import requests
import xml.etree.ElementTree as ET
import urllib
from datetime import datetime
from pathlib import Path

In [2]:
url = 'https://parlzhcdws.cmicloud.ch/parlzh2/cdws/Index/MITGLIEDER/searchdetails?q=seq>0 and %s and gremium all KR and funktion any "Mitglied Präsidium Vizepräsidium" sortBy name/sort.ascending vorname/sort.ascending&l=de-CH'

# from to
param = 'dauer_end >= "2010-11-19 00:00:00" and dauer_start <= "2022-12-31 00:00:00"'

# Get XML
r = requests.get(url % urllib.parse.quote(param))

# Load XML
tree = ET.ElementTree(ET.fromstring(r.text))
root = tree.getroot()

In [115]:
url % urllib.parse.quote(param)

'https://parlzhcdws.cmicloud.ch/parlzh2/cdws/Index/MITGLIEDER/searchdetails?q=seq>0 and dauer_end%20%3E%3D%20%222010-11-19%2000%3A00%3A00%22%20and%20dauer_start%20%3C%3D%20%222022-12-31%2000%3A00%3A00%22 and gremium all KR and funktion any "Mitglied Präsidium Vizepräsidium" sortBy name/sort.ascending vorname/sort.ascending&l=de-CH'

In [7]:
# Parse XML
ns = {
    's': 'http://www.cmiag.ch/cdws/searchDetailResponse',
    'd': 'http://www.cmiag.ch/cdws/SitzungenDetail',
    'm': 'http://www.cmiag.ch/cdws/Mitglieder'
    }

def str2dt(s):
    if s == '9999-12-31T23:59:59.000':
        s = '2100-12-31T00:00:00.000'
    return datetime.strptime(s, '%Y-%m-%dT00:00:00.000')

records = []

for mitglied in root.findall('.//m:Mitglied', ns):
    person = mitglied.find('m:Person', ns)

    end_person = mitglied.find('m:Dauer/m:End', ns).text
    if end_person == '9999-12-31T23:59:59.000':
            # Set 2100 als enddate
            end_person = '2100-12-31T00:00:00.000'

    start_person = datetime.strptime(mitglied.find('m:Dauer/m:Start', ns).text, '%Y-%m-%dT00:00:00.000')


    # Loop for each Party. If someone switch party, add person twice
    for party in person.findall('m:Kontakt/m:Parteizugehoerigkeiten/m:Parteizugehoerigkeit', ns):
        end = party.find('m:Dauer/m:End', ns).text
        if end == '9999-12-31T23:59:59.000':
            # Set end_person as end_date
            end = end_person
        end = datetime.strptime(end, '%Y-%m-%dT00:00:00.000')

        start = datetime.strptime(party.find('m:Dauer/m:Start', ns).text, '%Y-%m-%dT00:00:00.000')
        if start < start_person:
            start = start_person

        record = {
            'name': person.find('m:Kontakt/m:Name', ns).text,
            'vorname': person.find('m:Kontakt/m:Vorname', ns).text,
            'geschlecht': person.find('m:Kontakt/m:Geschlecht', ns).text,
            'wahlkreis': person.find('m:Kontakt/m:Wahlkreis', ns).text,
            'partei': party.find('m:Name', ns).text,
            'start': start,
            'end': end,
        }

        records.append(record)

    #print(datetime.strptime(mitglied.find('m:Dauer/m:End', ns).text, '%Y-%m-%dT00:00:00.000'))
    #break

df = pd.DataFrame(records)

# Save
df.to_csv(Path('../export/mitglieder.csv'), index=False)

## Download Präsidium
**!!!!! CORRUPT CSV!!!!!!!**

In [47]:
# ANPASSEN! https://raw.githubusercontent.com/openZH/krdb-lod/master/data/funktionen.csv
df_funktion = pd.read_csv('/Users/simon/Downloads/funktionen.csv', encoding='utf-8')
#df_person = pd.read_csv('https://raw.githubusercontent.com/openZH/krdb-lod/master/data/personen.csv')
df_person = pd.read_csv('/Users/simon/Downloads/personen.csv')

# Filter
df_funktion = df_funktion[df_funktion.FUNKTIONSBEZEICHNUNG.isin(['1. Vizepräsidium', 'Präsidium', '2. Vizepräsidium'])]

In [52]:
df_merged = df_funktion.merge(df_person, on = 'ID_PERSON_NEW')

df_merged['start'] = df_merged.apply(lambda row: "%s-%2d-%2d" % (row['DATUM_VON_JAHR'], row['DATUM_VON_MONAT'], row['DATUM_VON_TAG']), axis=1)
df_merged['end'] = df_merged.apply(lambda row: "%s-%2d-%2d" % (row['DATUM_BIS_JAHR'], row['DATUM_BIS_MONAT'], row['DATUM_BIS_TAG']), axis=1)

df_merged = df_merged[['VORNAME', 'NACHNAME', 'GESCHLECHT', 'FUNKTIONSBEZEICHNUNG', 'start', 'end']]
df_merged.rename(columns={'VORNAME': 'vorname', 'NACHNAME': 'nachname', 'GESCHLECHT': 'geschlecht', 'FUNKTIONSBEZEICHNUNG': 'funktion'}, inplace=True)

df_merged.to_csv(Path('../export/funktion.csv'), index=False)

,VORNAME,NACHNAME,GESCHLECHT,FUNKTIONSBEZEICHNUNG
0,Albert L.,Petermann,m,NaN
1,Jacques,Ruedin,m,NaN
2,Ernst,Nievergelt,m,NaN
3,Hansheinrich,Reimann,m,NaN
4,Otto,Fischer,m,NaN


## DEPRICATED

In [18]:
presidency = []

for person in root.findall('.//m:Mitglied/m:Person/m:Kontakt', ns):
    for mandat in person.findall('m:Behordenmandate/m:Behoerdenmandat', ns):
        if mandat.find('m:Kurzname', ns).text != 'GLKR':
            continue

        presidency.append({
            'vorname': person.find('m:Vorname', ns).text,
            'name': person.find('m:Name', ns).text,
            'funktion': mandat.find('m:Funktion', ns).text,
            'start': mandat.find('m:Dauer/m:Start', ns).text,
            'end': mandat.find('m:Dauer/m:End', ns).text,
        })
        
print(presidency)

[{'vorname': 'Markus', 'name': 'Bischoff', 'funktion': 'Mitglied', 'start': '2015-05-18T00:00:00.000', 'end': '2019-05-06T00:00:00.000'}, {'vorname': 'Markus', 'name': 'Bischoff', 'funktion': 'Mitglied', 'start': '2019-05-06T00:00:00.000', 'end': '9999-12-31T23:59:59.000'}, {'vorname': 'Yvonne', 'name': 'Bürgin', 'funktion': '1. Vizepräsidium', 'start': '2017-05-08T00:00:00.000', 'end': '2018-05-08T00:00:00.000'}, {'vorname': 'Yvonne', 'name': 'Bürgin', 'funktion': '2. Vizepräsidium', 'start': '2016-05-09T00:00:00.000', 'end': '2017-05-08T00:00:00.000'}, {'vorname': 'Yvonne', 'name': 'Bürgin', 'funktion': 'Präsidium', 'start': '2018-05-07T00:00:00.000', 'end': '2019-05-06T00:00:00.000'}, {'vorname': 'Yvonne', 'name': 'Bürgin', 'funktion': 'Mitglied', 'start': '2019-05-06T00:00:00.000', 'end': '9999-12-31T23:59:59.000'}, {'vorname': 'Barbara', 'name': 'Bussmann', 'funktion': 'Mitglied', 'start': '2010-05-03T00:00:00.000', 'end': '2011-05-09T00:00:00.000'}, {'vorname': 'Barbara', 'name':

In [ ]:
# Test

In [99]:
d = datetime.strptime('2019-03-01', '%Y-%m-%d')
d

datetime.datetime(2019, 3, 1, 0, 0)

In [103]:

x = df[(df.start <= d) & (df.end >= d)]
len(x)

179

In [107]:
x.sort_values('name')[20:30]

,name,vorname,geschlecht,wahlkreis,partei,start,end
34,Brazerol,Rico,männlich,IX Horgen,BDP,2011-05-09,2019-05-06
35,Brunner,Hans-Peter,männlich,IX Horgen,FDP,2015-05-18,2100-12-31
33,Brändli,Roland,männlich,XI Hinwil,SVP,2019-01-01,2019-05-06
39,Burtscher,Rochus,männlich,VII Dietikon,SVP,2011-05-09,2100-12-31
41,Bussmann,Barbara,weiblich,XII Uster,SP,2009-01-01,2019-05-06
14,Bär,Hansruedi,männlich,VI Zürich 11+12,SVP,2009-01-01,2019-05-06
18,Bärtschiger,Markus,männlich,VII Dietikon,SP,2017-05-15,2100-12-31
38,Bürgin,Yvonne,weiblich,XI Hinwil,Die Mitte,2013-08-19,2100-12-31
43,Bütikofer,Kaspar,männlich,II Zürich 3+9,AL,2009-01-01,2021-09-06
44,Camenisch,Linda,weiblich,XVII Bülach,FDP,2010-08-23,2100-12-31


In [108]:
df[df.vorname == 'Robert']

,name,vorname,geschlecht,wahlkreis,partei,start,end
36,Brunner,Robert,männlich,XVIII Dielsdorf,Grüne,2020-02-01,2020-02-11


In [92]:
x['dup'] = x.duplicated(subset=['name', 'vorname'], keep=False)
x.head()

/var/folders/xv/8dpn0yrd2_sdyj5_kf48mxbh0000gn/T/ipykernel_27588/2518723928.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x['dup'] = x.duplicated(subset=['name', 'vorname'], keep=False)


,name,vorname,geschlecht,wahlkreis,partei,start,end,dup
0,Ackermann,Pia,weiblich,II Zürich 3+9,SP,2015-05-18,2100-12-31,False
1,Ackermann,Ruth,weiblich,VI Zürich 11+12,Die Mitte,2015-05-18,2100-12-31,False
3,Agosti Monn,Theres,weiblich,XV Winterthur Land,SP,2013-08-19,2100-12-31,False
5,Albanese,Franco,männlich,XIV Winterthur Stadt,SVP,2016-09-06,2020-08-15,False
7,Alder,Ronald,männlich,VIII Affoltern,GLP,2017-08-21,2100-12-31,False


In [93]:
x[x.dup == True]

,name,vorname,geschlecht,wahlkreis,partei,start,end,dup


In [13]:
len(root.findall('.//m:Person', ns))

294